In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install pytorch-lightning

In [3]:
import pytorch_lightning as pl
import pandas as pd
import cv2
import os 
import torchvision
from torch import nn
from torch.utils.data import Dataset ,DataLoader, random_split
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import transforms, datasets, models
import matplotlib.pyplot as plt
import torchmetrics
from torchmetrics.functional import accuracy
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks.progress import TQDMProgressBar
from pytorch_lightning.loggers import CSVLogger
from torchvision.utils import make_grid
import math
import torch.nn.functional as F
from torchvision.ops import box_convert

In [4]:
from PIL import Image
from pycocotools.coco import COCO
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [5]:
class myOwnDataset(Dataset):
  def __init__(self, root, annotation, transforms=None):
    self.root = root
    self.transforms = transforms
    self.coco = COCO(annotation)
    self.ids = list(sorted(self.coco.imgs.keys()))

  def __getitem__(self, idx):
    # own coco file
    coco = self.coco
    # image ID
    img_id = self.ids[idx]
    # List: get annotation id from coco
    ann_ids = coco.getAnnIds(imgIds=img_id)
    # Dictionary: target coco_annotation file for an image
    coco_annotation = coco.loadAnns(ann_ids)
    # path for input image
    path = coco.loadImgs(img_id)[0]["file_name"]
    # open the input image
    img = Image.open(os.path.join(self.root, path))
    # number of objects in the image
    num_objs = len(coco_annotation)

    # bounding boxes for objects 
    # In coco format, bbox = [xmin, ymin, width, height]
    # In pytorch, the input should be [xmin, ymin, xmax, ymax]
    boxes = []
    for i in range(num_objs):
      xmin = coco_annotation[i]["bbox"][0]
      ymin = coco_annotation[i]["bbox"][1]
      width = coco_annotation[i]["bbox"][2]
      height = coco_annotation[i]["bbox"][3]
      boxes.append([xmin, ymin, width, height])
    boxes = torch.Tensor(boxes)
    # convert box format from "xywh" to "xyxy"
    boxes = box_convert(
                boxes, in_fmt="xywh", out_fmt="xyxy"
            )
    boxes = torch.as_tensor(boxes, dtype=torch.float32)
    # labels (In my case, I only one class: target class or background)
    labels = torch.ones((num_objs, ), dtype=torch.int64)
    # tensorize img_id
    img_id = torch.tensor([img_id])
    # size of bbox (rectangular)
    areas = []
    for i in range(num_objs):
      areas.append(coco_annotation[i]["area"])
    areas = torch.as_tensor(areas, dtype=torch.float32)
    # iscrowd
    iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

    # Annotation is in dictionary format
    my_annotation = {}
    my_annotation["boxes"] = boxes
    my_annotation["labels"] = labels
    my_annotation["image_id"] = img_id
    my_annotation["area"] = areas
    my_annotation["iscrowd"] = iscrowd

    if self.transforms is not None:
      img = self.transforms(img)

    return img, my_annotation

  def __len__(self):
    return len(self.ids)

In [16]:
  # collate_fn needs for batch
  def custom_collate_fn(self, batch):
    return tuple(zip(*batch))

In [17]:
def get_model_instance_segmentation(num_classes):
  # load instance segmentation model pre-trained on COCO
  model = models.detection.fasterrcnn_resnet50_fpn(weights=None)
  # get number of input features for the classifier
  in_features = model.roi_heads.box_predictor.cls_score.in_features
  # replace the pre-trained head with a new one
  model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

  return model

In [18]:
# path to my data and coco file
train_data_dir = "/content/drive/MyDrive/Datasets/my_data/train"
train_coco = "/content/drive/MyDrive/Datasets/my_data/my_train_coco.json"

# batch size
train_batch_size = 1

# params for dataloader
train_shuffle_dl = True
num_workers_dl = 4

# params for training

# two classes; only target class and background
num_classes = 2
num_epochs = 10

lr = 0.005
momentum = 0.9
weight_decay = 0.005

In [19]:
from torchvision.models import detection
class LitDataModule(pl.LightningDataModule):
  def __init__(self):
    super().__init__()
    self.save_hyperparameters()
    #self.transform = self.get_transform()
    self.transform = transforms.ToTensor()

  def get_transform(self):
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return transforms.Compose(custom_transforms)

  def setup(self, stage=None):
    self.dataset = myOwnDataset(root=train_data_dir, annotation=train_coco, transforms=self.transform)

  def train_dataloader(self):
    return DataLoader(self.dataset,
                      batch_size=train_batch_size,
                      shuffle=train_shuffle_dl,
                      num_workers=num_workers_dl,
                      collate_fn=custom_collate_fn,)


In [20]:
dm = LitDataModule()
dm.setup()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [21]:
class LitModel(pl.LightningModule):
  def __init__(self, dataset):
    super().__init__()
    self.dataset = dataset
    self.model = get_model_instance_segmentation(2)
    
  def forward(self, x):
    x = self.model(x)
    return x

  def training_step(self, batch, batch_idx):
    imgs, annotations = batch
    loss_dict = self.model(imgs, annotations)
    loss = sum(loss for loss in loss_dict.values())
    self.log("train loss", loss)
    return loss

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [22]:
model = LitModel(dm.dataset)
trainer = pl.Trainer(max_epochs=num_epochs,
                  accelerator="auto",
                  devices=1 if torch.cuda.is_available() else None,
                  callbacks=[LearningRateMonitor(logging_interval="step"),
                                TQDMProgressBar(refresh_rate=20)],
)
trainer.fit(model, dm)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1898: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

TypeError: ignored